<a href="https://colab.research.google.com/github/Arvin-007Elite/PYTHON/blob/master/Text_Data_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import re
import nltk
# means regular expression library
from nltk.corpus import stopwords
# nltk  is natural language toolkit

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# printing  stopwards
print(stopwords.words('english'))
# these words  can be repeated many times in our dataset , which is unwanted , and has no meaning ,
# so we have to remove these , so stopwords library is used

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# data pre-preprocessing
news_data = pd.read_csv('/content/news.csv')

In [9]:
news_data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
# if  label 0 means news is real , if 1 its fake
news_data.shape

(6335, 4)

In [11]:

# if datset is large , prediction will be accurate and performance will be good
news_data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [16]:
!pip install faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00


In [20]:
from faker import Faker


# Assuming you have a DataFrame named news_data with a 'title' column
fake = Faker()
news_data['author'] = [fake.name() for _ in range(len(news_data))]

In [21]:
# if null values r there , we can replace it with null string , we cant mean and model as it is not numbers
# here theres no null , but will practise code

# news_data = news_data.fillna('')


# it replace null as empty  brackets

# will create separate col as author + title
# merging starts

news_data['content'] = news_data['author'] + ' ' + news_data['title']


# new col name is content in that  author  + space + title is merged

In [22]:
# now will do text processing

# separating content & label , which is feature and target

news_data.head()

,Unnamed: 0,title,text,label,author,content
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,Steven Sheppard,Steven Sheppard You Can Smell Hillary’s Fear
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,Anthony Pierce,Anthony Pierce Watch The Exact Moment Paul Rya...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,Cole Gray,Cole Gray Kerry to go to Paris in gesture of s...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,Melissa Cortez,Melissa Cortez Bernie supporters on Twitter er...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,Michelle Mora,Michelle Mora The Battle of New York: Why This...


In [25]:
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']

In [26]:
print(Y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object


In [27]:
# Y is already in the form of labels and numeric
# so x needs to converted like Y

# we use Stemming for tht
# it is process of reducing the word to its Rootword
# eg : enjoyment -> enjoy
# thats why stemmer package is used !

In [56]:
port_stem = PorterStemmer()

# we r removing  waste words which is stopwords  and if it big wate word , it will shrinked to root word

# Define stemming function
def stemming(content):
    # Convert integers to string
    content = str(content)
    # Apply stemming
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

# Apply stemming function to the 'content' column
news_data['content'] = news_data['content'].apply(stemming)

In [53]:
news_data.dtypes

Unnamed: 0     int64
title         object
text          object
label         object
author        object
content       object
dtype: object

In [57]:
# now we applied stemming todatset


In [58]:
# we use  faeture extraction to convert into vectors

print(news_data['content'])

0                      steven sheppard smell hillari fear
1       anthoni pierc watch exact moment paul ryan com...
2                 cole gray kerri go pari gestur sympathi
3       melissa cortez berni support twitter erupt ang...
4               michel mora battl new york primari matter
                              ...                        
6330    derek bailey state depart say find email clint...
6331         annett skinner p pb stand plutocrat pentagon
6332    victoria brown anti trump protest tool oligarc...
6333    joseph collin ethiopia obama seek progress pea...
6334    lindsey shaw jeb bush suddenli attack trump ma...
Name: content, Length: 6335, dtype: object


In [59]:
# everything is shortened
X = news_data['content'].values
Y  = news_data['label'].values

In [60]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [61]:
Y.shape

(6335,)

In [62]:
# now ill aplly to feature vectors
# textual data to feature vectors

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [63]:
print(X)

  (0, 6671)	0.3980347483440816
  (0, 6455)	0.5658958367604031
  (0, 6297)	0.5425172723561593
  (0, 3217)	0.23442195687840897
  (0, 2515)	0.41479102735767687
  (1, 7580)	0.25577316120100557
  (1, 7470)	0.19959034678162862
  (1, 7198)	0.1245271681674199
  (1, 6779)	0.3215409852500123
  (1, 6034)	0.24165622066658235
  (1, 5628)	0.26978978575152823
  (1, 5346)	0.22333230680917726
  (1, 5272)	0.33243966103349903
  (1, 5163)	0.23005445206786318
  (1, 4573)	0.3011147233925136
  (1, 2385)	0.4094812031195801
  (1, 1405)	0.32489009734557023
  (1, 271)	0.2597054082543004
  (2, 6869)	0.4603486261812745
  (2, 5121)	0.32392833489866446
  (2, 3832)	0.3434862733314828
  (2, 2972)	0.3594477408740005
  (2, 2916)	0.2891483736875344
  (2, 2864)	0.4603486261812745
  (2, 1365)	0.37474849993440446
  :	:
  (6332, 7198)	0.1539433564402988
  (6332, 7086)	0.4674854943174906
  (6332, 5517)	0.3015349640433885
  (6332, 4950)	0.4448330495229588
  (6332, 3483)	0.3495925188933479
  (6332, 924)	0.2870980706378511
  (63

In [64]:
# even x is converted in nimbers
# nxt train , test , split and feed it to machine